## Homework: Agents

In this homework, we will learn more about function calling,
and we will also explore MCP - model-context protocol. 


## Preparation

First, we'll define a function that we will use when building
our agent. 

It will generate fake weather data:

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description

We want to use it as a tool for our agent, so we need to 
describe it 

How should the description for this function look like? Fill in missing parts

```python
get_weather_tool = {
    "type": "function",
    "name": "<TODO1>",
    "description": "<TODO2>",
    "parameters": {
        "type": "object",
        "properties": {
            "<TODO3>": {
                "type": "string",
                "description": "<TODO4>"
            }
        },
        "required": [TODO5],
        "additionalProperties": False
    }
}
```

What did you put in `TODO3`?

> `city`

In [2]:
get_weather_tool = {
    "type": "function",                       # ← tells the model this is a callable tool
    "name": "get_weather",                    # TODO1
    "description": "Return the current temperature in °C for a given city.",  # TODO2
    "parameters": {
        "type": "object",
        "properties": {
            "city": {                         # TODO3
                "type": "string",
                "description": "Name of the city to retrieve the weather for."  # TODO4
            }
        },
        "required": ["city"],                 # TODO5
        "additionalProperties": False
    }
}


***Why these choices***

| Field                         | Value chosen                         | Reason                                                                                                                |
| ----------------------------- | ------------------------------------ | --------------------------------------------------------------------------------------------------------------------- |
| `name`                        | `"get_weather"`                      | Must exactly match your Python function’s name so the orchestrator can link the model’s call to the code.             |
| `description`                 | Clear, one-sentence goal (“Return…”) | The model uses this natural-language text to decide *when* and *how* to call the tool. Clarity reduces hallucination. |
| `properties.city`             | `"city"`                             | The sole input your function needs. Keeping the schema minimal prevents the model from inventing extra keys.          |
| `properties.city.description` | Explains the value                   | Helps the model format the argument correctly (plain city name, no extra stuff).                                      |
| `required`                    | `["city"]`                           | Marks the key as mandatory; the model won’t try to omit or rename it.                                                 |
| `additionalProperties`        | `False`                              | Blocks unexpected keys (e.g., `"country"`, `"unit"`), which could crash your function.                                |


## Testing it (Optional)

If you have OpenAI API Key (or alternative provider), let's test it.

A question could be `"What's the weather like in Germany?"`

Experiment with different system prompts to have better answers
from the system.

You can use [chat_assistant.py](https://github.com/alexeygrigorev/rag-agents-workshop/blob/main/chat_assistant.py)
or implement everything yourself 


In [4]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-22 21:08:13--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3485 (3,4K) [text/plain]
Saving to: ‘chat_assistant.py’

chat_assistant.py   100%[===================>]   3,40K  --.-KB/s    in 0s      

2025-07-22 21:08:14 (21,0 MB/s) - ‘chat_assistant.py’ saved [3485/3485]



Create [run_chat.py](./run_chat.py) and run it with `python run_chat.py`.

## Q2. Adding another tool

Let's add another tool - a function that can add weather data
to our database:

In [5]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

Now let's write a description for it.

What did you write?

> Add or update the current temperature in °C for a given city.

Optionally, you can test it after adding this function.

In [7]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Add or update the current temperature in °C for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to store the temperature for."
            },
            "temp": {
                "type": "number",
                "description": "Current temperature of the city in degrees Celsius."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}


***Explanation***

| Field                  | Value                                  | Reason                                                                       |
| ---------------------- | -------------------------------------- | ---------------------------------------------------------------------------- |
| `name`                 | `"set_weather"`                        | Must match the actual Python function you’ll call.                           |
| `description`          | Clear, imperative sentence             | Helps the LLM decide *when* to use the tool and reminds it of the unit (°C). |
| `properties.city`      | `"string"` + city-specific description | Lets the model know it should pass a plain text city name.                   |
| `properties.temp`      | `"number"` + °C note                   | Accepts floats (e.g. `21.4`) and makes the expected unit explicit.           |
| `required`             | `["city", "temp"]`                     | Both keys are mandatory; prevents calls with missing data.                   |
| `additionalProperties` | `False`                                | Stops the model from inventing extra keys like `country`.                    |


## MCP

MCP stands for Model-Context Protocol. It allows LLMs communicate
with different tools (like Qdrant). It's function calling, but 
one step further:

* A tool can export a list of functions it has
* When we include the tool to our Agent, we just need to include the link to the MCP server

## Q3. Install FastMCP

Let's install a library for MCP - [FastMCP](https://github.com/jlowin/fastmcp):

What's the version of FastMCP you installed?

> 2.10.6

In [8]:
!pip install fastmcp

  Using cached exceptiongroup-1.3.0-py3-none-any.whl.metadata (6.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached docutils-0.21.2-py3-none-any.whl.metadata (2.8 kB)
Using cached exceptiongroup-1.3.0-py3-none-any.whl (16 kB)
Using cached docutils-0.21.2-py3-none-any.whl (587 kB)
  DEPRECATION: Building 'pyperclip' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pyperclip'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11004 sha256=0aeb77f4983b9fa9d12149c1b1c4eb35195c7cb9a831851ac8f2f4e03f4ff1f3
  Stored in directory: /Users/gabi/Library/Caches/pip/wheels/cc/ae/36/ee17d1de094fcb61e241

In [10]:
import fastmcp

print("FastMCP version:", fastmcp.__version__)

FastMCP version: 2.10.6


## Q4. Simple MCP Server

A simple MCP server from the documentation looks like that:

```python
# weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()
```

In our case, we need to write docstrings for our functions.

Let's ask ChatGPT for help:

```python
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'
```

Let's change the example for our case and run it

What do you see in the output?

Look for a string that matches this template:

`Starting MCP server 'Demo 🚀' with transport '<TODO>'`

What do you have instead of `<TODO>`?

> *Starting MCP server 'Demo 🚀' with transport `'stdio'`*

## Q5. Protocol

There are different ways to communicate with an MCP server.
Ours is currently running using standart input/output, which
means that the client write something to stdin and read the
answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:

- First, we send an initialization request -- this way, we register our client with the server:
    ```json
    {"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
    ```
    - We should get back something like that, which is an aknowledgement of the request:
    ```json
    {"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.9.4"}}}
    ```
-  Next, we reply back, confirming the initialization:
    ```json
    {"jsonrpc": "2.0", "method": "notifications/initialized"}
    ```
    - We don't expect to get anything in response
- Now we can ask for a list of available methods:
    ```json
    {"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
    ```
- Let's ask the temperature in Berlin:
    ```json
    {"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": { "city": "Berlin" } }}
    ```
- What did you get in response?

> {"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"24.3"}],"structuredContent":{"result":24.3},"isError":false}}

*Explanation*:

- run the server with `python weather_server.py` and then send the requests from the same shell. 

## Q6. Client

We typically don't interact with the server by copy-pasting 
commands in the terminal.

In practice, we use an MCP Client. Let's implement it. 

FastMCP also supports MCP clients:

```python
from fastmcp import Client

async def main():
    async with Client(<TODO>) as mcp_client:
        # TODO
```

Use the client to get the list of available tools
of our script. How does the result look like?

> [Tool(name='get_weather', title=None, description='Retrieve the current temperature for *city*.\n\nParameters\n----------\ncity : str\n    Name of the city (case-insensitive).\n\nReturns\n-------\nfloat\n    Temperature in °C.  If the city was never written before,\n    a random placeholder is generated so we always return a number.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description='Store or update the temperature for *city*.\n\nParameters\n----------\ncity : str\n    Name of the city (case-insensitive).\ntemp : float\n    Temperature in °C to associate with the city.\n\nReturns\n-------\nstr\n    Always "OK" to signal success.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}}, 'required': ['city', 'temp'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None)]


If you're running this code in Jupyter, you need to pass
an instance of MCP server to the `Client`:

```python
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # ....
```

If you run it in a script, you will need to use asyncio:

```python
import asyncio

async def main():
    async with Client("weather_server.py") as mcp_client:
        # ...

if __name__ == "__main__":
    test = asyncio.run(main())
```

Copy the output with the available tools when
filling in the homework form.

In [16]:
import asyncio
from fastmcp import Client
import weather_server          # only needed in-notebook

async def main():
    # notebook → pass the server object
    async with Client(weather_server.mcp) as client:
        # 1️⃣ list the tools the server exposes
        tools = await client.list_tools()          # <— new name
        print("Available tools - homework answer:")
        print("\n|---------------------------|\n")
        print (tools)
        print("\n|---------------------------|\n")
        for tool in tools:
            print(tool.name, "→", tool.description)

        # 2️⃣ (optional) call one
        berlin = await client.call_tool(           # <— new name
            "get_weather", {"city": "Berlin"}
        )
        print("Berlin temp:", berlin.data)         # .data unwraps the primitive

# notebook: just await
await main()


Available tools - homework answer:

|---------------------------|

[Tool(name='get_weather', title=None, description='Retrieve the current temperature for *city*.\n\nParameters\n----------\ncity : str\n    Name of the city (case-insensitive).\n\nReturns\n-------\nfloat\n    Temperature in °C.  If the city was never written before,\n    a random placeholder is generated so we always return a number.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description='Store or update the temperature for *city*.\n\nParameters\n----------\ncity : str\n    Name of the city (case-insensitive).\ntemp : float\n    Temperature in °C to associate with the city.\n\nReturns\n-------\nstr\n    Al